<a href="https://colab.research.google.com/github/mickymult/RAG-ChromaDB-Mistral7B/blob/main/RAG_Chromadb_mistral7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Mar  7 20:51:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q pypdf python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
!pip install -q sentence_transformers
!pip install -q llama-index
!pip install -q chromadb
!pip install -q llama-index-embeddings-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q llama-index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, VectorStoreIndex

from llama_index.llms.huggingface import HuggingFaceLLM

from langchain.embeddings import HuggingFaceEmbeddings

from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb

from IPython.display import Markdown, display

In [ ]:
chroma_client = chromadb.PersistentClient(path='/content/drive/MyDrive/pdf_documents/chroma_persist')

In [ ]:
# chroma_client = chromadb.PersistentClient()
# chroma_collection = chroma_client.create_collection('climate_report')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/pdf_documents").load_data()

In [ ]:
# from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [ ]:
# !huggingface-cli login

In [ ]:
import torch
torch.set_default_device('cuda')

In [ ]:
llm = HuggingFaceLLM(
    context_window=8000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    tokenizer_kwargs={"max_length": 8000},
    model_kwargs={"torch_dtype": torch.float16}

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
embed_model =  HuggingFaceEmbeddings(model_name="thenlper/gte-base")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-16-6394536e40c5>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
import sqlite3

# Establish a connection and create a cursor
conn = sqlite3.connect('/content/drive/MyDrive/pdf_documents/chroma.sqlite3')
cursor = conn.cursor()
print(cursor)
# Execute a query to retrieve the data you need
cursor.execute("SELECT document_content FROM documents")
documents = [row[0] for row in cursor.fetchall()]


OperationalError: no such table: documents

In [ ]:
import sqlite3

db_path = '/content/drive/MyDrive/pdf_documents/chroma.sqlite3'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# Example table creation for documents - adjust according to your actual data structure
cursor.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id INTEGER PRIMARY KEY,
        content TEXT
    )
""")
conn.commit()


In [ ]:
import sqlite3
import pickle  # Used for simplifying the serialization/deserialization example

# Connect to your SQLite database
conn = sqlite3.connect('/content/drive/My Drive/pdf_documents/chroma.sqlite3')
cursor = conn.cursor()

# Function to check if a vector exists and return it
def get_vector_for_document(document_id):
    cursor.execute("SELECT vector FROM vectors WHERE document_id = ?", (document_id,))
    result = cursor.fetchone()
    if result:
        return pickle.loads(result[0])  # Deserializing the vector
    return None

# Function to save a new vector for a document
def save_vector_for_document(document_id, vector):
    serialized_vector = pickle.dumps(vector)  # Serializing the vector
    cursor.execute("INSERT INTO vectors (document_id, vector) VALUES (?, ?)", (document_id, serialized_vector))
    conn.commit()

# Example usage
document_id = 1  # Assuming this is the ID of the document you're working with
vector = get_vector_for_document(document_id)
if vector is None:
    # Replace 'your_document_content_here' with the actual content or how you retrieve it
    vector = generate_vector('your_document_content_here')  # You need to define this function
    save_vector_for_document(document_id, vector)
else:
    print("Vector already exists, using the existing vector.")


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("what is air bnb?")
display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Airbnb is a platform that allows people to rent out their homes, apartments, and other types of properties to travelers. The company was founded in 2008 by three friends, Brian Chesky, Joe Gebbia, and Nathan Blecharczyk, and has since grown to become one of the largest online travel platforms in the world. Airbnb operates in over 220 countries and territories, and has over 150 million listings. The company takes a 10% commission on each transaction and generates revenue through fees charged to both hosts and travelers.</b>

In [ ]:
response = query_engine.query("give me summary of ubercab")
display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>UberCab is an on-demand car service that offers luxury service to its customers. It has a modern and fuel-efficient fleet and is customer-focused, with a computer-coordinated system. UberCab aims to provide the best end-user experience possible with a statistically optimized response time. It has a pre-paid, cashless billing system and is profitable by design. The concept of UberCab is to provide a fast and efficient on-demand car service to professionals in American cities, offering the convenience of a cab in NYC combined with the experience of a professional chauffeur. UberCab uses the latest consumer web and device technology to automate dispatch and reduce wait times. It optimizes its fleets and incentivizes drivers to provide the "NetJets of car services".</b>

In [ ]:
response = query_engine.query("How many properties stolen by china alone?")
display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Every year, $200-$500 billion in American intellectual property is stolen by China alone.</b>

In [ ]:
response = query_engine.query("what is the solution proposed by sicura?")
display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>The solution proposed by Sicura is to develop secure infrastructure-as-code that can be easily deployed across hybrid infrastructure. This solution aims to increase deployment speed and custom security profiles for specific needs. The result of this solution is improved efficiency by maintaining consistent security posture across isolated environments.</b>

In [ ]:
response = query_engine.query("give me the funding story year by year of sicura")
display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Sicura's funding story year by year is as follows:

* In 2009, Sicura spun out from the NSA and received seed funding.
* In 2015, the software was open-sourced for non-NSA users.
* In 2016, Sicura landed its first enterprise customer.
* In 2023, Sicura raised $4M to fund and execute its go-to-market plan.

It's worth noting that Sicura's revenue has been growing steadily, with an ARR of $1M in Q4 2023 and a plan to exceed $2M ARR by the end of 2024.</b>